In [10]:
import pandas as pd
import re
ratings_movies = pd.read_csv('data/ratings_movies.csv', sep=',')
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [ ]:
# 8.1
#Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
#У скольких фильмов не указан год их выпуска?
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


-18

In [13]:
# 8.2
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
mask = ratings_movies['year_release'] == 1999
ratings_movies[mask].groupby('title')['rating'].mean().sort_values()


title
Bloodsport: The Dark Kumite (1999)                  0.5
Trippin' (1999)                                     1.0
Chill Factor (1999)                                 1.0
From Dusk Till Dawn 2: Texas Blood Money (1999)     1.0
Simon Sez (1999)                                    1.0
                                                   ... 
Mickey's Once Upon a Christmas (1999)               5.0
On the Ropes (1999)                                 5.0
Trailer Park Boys (1999)                            5.0
Five Senses, The (1999)                             5.0
George Carlin: You Are All Diseased (1999)          5.0
Name: rating, Length: 261, dtype: float64

In [14]:
# 8.3
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
mask1 = ratings_movies['year_release'] == 2010
ratings_movies[mask1].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Comedy|Musical                       5.000000
Name: rating, Length: 119, dtype: float64

In [22]:
# 8.4
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов? 
ratings_movies.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
494     15
578     15
214     13
85      13
245     13
Name: genres, Length: 610, dtype: int64

In [24]:
# 8.5
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
ratings_movies.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(by='count', ascending=False)

,count,mean
userId,,
414,2698,3.391957
599,2478,2.642050
474,2108,3.398956
448,1864,2.847371
274,1346,3.235884
...,...,...
442,20,1.275000
278,20,3.875000
147,20,3.375000


In [34]:
mask3 = ratings_movies['year_release'] == 2018
table = ratings_movies[mask3].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
mask4 = table['count'] > 10
table[mask4].sort_values(by='mean', ascending=False)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [47]:
# 8.7
# Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную таблицу, 
# которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
table2 = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres'
)
table2['Comedy']

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64

In [65]:
orders = pd.read_csv('data/orders.csv', sep=';')
products = pd.read_csv('data/products.csv', sep=';')
orders_products = orders.merge(
    products,
    left_on='ID товара',
    right_on='Product_ID',
    how='left'
)
display(orders_products)

,Дата создания,Order ID,ID Покупателя,Статус,Оплачен,Отменен,Отгружен,ID товара,Количество,Product_ID,Name,Price,CURRENCY
0,09.11.2019 21:55:51,9,10,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
1,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,86,100,86.0,"Носки Простые, муж",45.0,RUR
2,09.11.2019 15:05:57,8,9,"Принят, ожидается оплата",Нет,Нет,Нет,104,10,104.0,"Носки Подарочные, жен",249.0,RUR
3,09.11.2019 12:50:07,7,8,"Принят, ожидается оплата",Нет,Нет,Нет,104,7,104.0,"Носки Подарочные, жен",249.0,RUR
4,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,104,5,104.0,"Носки Подарочные, жен",249.0,RUR
5,09.11.2019 12:00:00,6,1,"Принят, ожидается оплата",Нет,Нет,Нет,103,5,103.0,"Носки Подарочные, муж",199.0,RUR
6,08.11.2019 08:36:22,5,5,Отменён,Нет,Да,Нет,124,1,124.0,Носки беговые Camino,999.0,RUR
7,08.11.2019 08:36:22,4,9,"Принят, ожидается оплата",Нет,Нет,Да,91,1,91.0,"Носки Честные, муж",50.0,RUR
8,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,103,3,103.0,"Носки Подарочные, муж",199.0,RUR
9,08.11.2019 08:36:22,3,8,"Оплачен, формируется к отправке",Да,Нет,Нет,104,3,104.0,"Носки Подарочные, жен",249.0,RUR


In [66]:
orders_products['Отменен'] == 'Да'


0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
Name: Отменен, dtype: bool

In [68]:
# 8.10
# Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину за указанный период?
# Подсказка (1 из 3): Cоздайте признак прибыли за один заказ — profit — количество товара * цена товара.
# Подсказка (2 из 3): Произведите фильтрацию по условию оплаченного заказа.
# Подсказка (3 из 3): Сгруппируйте таблицу по столбцу с идентификатором покупателя и рассчитайте суммарную цену товаров, 
# заказанных ими. Отсортируйте результат по убыванию. Идентификатор покупателя в первой строке полученного топа — наш ответ.
mask5 = orders_products['Оплачен'] == 'Да'
orders_products['Profit'] = orders_products['Количество'] * orders_products['Price']
orders_products[mask5].groupby('ID Покупателя')['Profit'].sum().sort_values()

ID Покупателя
1        0.0
8     1344.0
5    13043.0
7    17096.0
Name: Profit, dtype: float64